In [1]:
import os
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm


In [ ]:
#Load predictions from Pickle dump
with open(f'predictions_15_03_2020.pickle', 'rb') as f:
    preds = pickle.load(f)
f.close()
with open(f'paths_15_03_2020.pickle', 'rb') as f:
    paths = pickle.load(f)
f.close()
paths = np.asarray(paths)
paths = np.concatenate(paths)
preds = np.asarray(preds)
preds = np.concatenate(preds)
data = np.stack((paths,preds))
#print(data[0].shape)
filt = list(filter(lambda x: 'M1' in x, data[0]))
preds = data[1][np.in1d(data[0], filt)]
print(len(preds))
print(preds)

In [12]:
def write_to_csv(values_df, filename, profile_n, fn):
    df = pd.read_csv(filename,sep=';')
    df["Filename"] = df["Filename"].astype(str)
    df["Filename"] = df["Filename"].apply(lambda x: os.path.join('/home/plankton/Results/Peru/crops', profile_n , x))
    df = df.merge(values_df, on='Filename')
    filename_n = os.path.join(profile_dir, 'results', fn)
    os.makedirs(os.path.join(profile_dir, 'results'),exist_ok=True)
    df.to_csv(filename_n, index=False, sep=';')

In [ ]:
data_dir = '/home/plankton/Results/Peru/crops_new/'
for day in os.listdir(data_dir):
    print(f'######## processing directory {day}...')
    day_dir = os.path.join(data_dir,day)
    #Load predictions from Pickle dump
    with open(f'predictions_{day}.pickle', 'rb') as f:
        preds = pickle.load(f)
    f.close()
    #Load paths from Pickle dump
    with open(f'paths_{day}.pickle', 'rb') as f:
        paths = pickle.load(f)
    f.close()
    paths = np.asarray(paths)
    paths = np.concatenate(paths)
    preds = np.asarray(preds)
    preds = np.concatenate(preds)
    data = pd.DataFrame({'Filename': paths, 'class': preds})
    for profile in os.listdir(day_dir):
        print(f'processing profile {day}:{profile}...')
        profile_dir = os.path.join(day_dir,profile)
        data_prof = data[data['Filename'].str.contains(profile)]
    #    filt = list(filter(lambda x: profile in x, data['Filename']))
    #    paths_prof = data['Filename'][np.in1d(data['Filename'], filt)]
    #    preds_prof = data['class'][np.in1d(data['class'], filt)]
    #    data_prof = pd.DataFrame({'Filename': paths_prof, 'class': preds_prof})
        for fn in tqdm(os.listdir(profile_dir)):
            if fn.endswith('.csv'):
                try:
                    profile_name = os.path.join(day,profile)
                    file = os.path.join(profile_dir, fn)
                    #print(f'processing file {file}...')
                    write_to_csv(data_prof,file,profile_name,fn)
                    #df = pd.read_csv(os.path.join(profile_dir,fn), delimiter=';')
                    #for i,k in zip(tqdm(paths_prof),range(len(paths_prof))):
                    #    df.loc[df['Filename'] == i, 'class'] = preds_prof[k]
                    #df.to_csv(fn, sep=';',index=False)
                    #print(f'wrote to file {fn}')
                except KeyError:
                    print(f'cannot write to {fn}')
                    pass


In [9]:
data_dir = '/home/plankton/Results/Peru/crops_new/'
for day in os.listdir(data_dir):
    print(f'######## processing directory {day}...')
    day_dir = os.path.join(data_dir,day)
    for profile in os.listdir(day_dir):
        print(f'processing profile {day}:{profile}...')
        meta_file = os.path.join(day_dir,profile,'metadata.csv')
        meta_df = pd.read_csv(meta_file, sep=';', names= ["Filename", "Bg_raw", "Bg_clean", "object_count"])
        profile_results_dir = os.path.join(day_dir,profile,'results')
        for img_info_f in os.listdir(profile_results_dir):
            img_info_path = os.path.join(profile_results_dir,img_info_f)
            img_path = img_info_f[:-3]+'tif'
            df = pd.read_csv(img_info_path,sep=';')
            copepod_counts = int(len(df[df['class'] == 0]))
            copepod_avg_area = df.loc[df['class']==0, ['area']].mean()
            meta_df.loc[meta_df['Filename']==img_path, ['avg_area']]=copepod_avg_area[0]
            meta_df.loc[meta_df['Filename']==img_path, ['copepod_count']]=copepod_counts
        output = os.path.join(day_dir,profile,'overview.csv')
        meta_df.to_csv(output, index=False, sep=';')



######## processing directory 13_03_2020...
processing profile 13_03_2020:M1...
processing profile 13_03_2020:M5...
processing profile 13_03_2020:M8...
processing profile 13_03_2020:PA...
processing profile 13_03_2020:M3...
processing profile 13_03_2020:M6...
processing profile 13_03_2020:M7...
processing profile 13_03_2020:M2...
processing profile 13_03_2020:M9...
processing profile 13_03_2020:M4...
######## processing directory 21_03_2020...
processing profile 21_03_2020:E8...
processing profile 21_03_2020:M1...
processing profile 21_03_2020:M5...
processing profile 21_03_2020:M8...
processing profile 21_03_2020:PA...
processing profile 21_03_2020:M3...
processing profile 21_03_2020:M6...
processing profile 21_03_2020:M7...
processing profile 21_03_2020:M2...
processing profile 21_03_2020:M9...
processing profile 21_03_2020:M4...
######## processing directory 25_03_2020...
processing profile 25_03_2020:M1...
processing profile 25_03_2020:M5...
processing profile 25_03_2020:M8...
proc

In [8]:
df = pd.read_csv('/home/plankton/Results/Peru/crops_new/17_03_2020/M3/results/CUSCO-Peru-2020_M3_PISCO20200317_151450.0700_1.215bar.csv',sep=';')
copepod_counts = len(df[df['class'] == 0])
copepod_avg_area = df.loc[df['class']==0, ['area']].mean()
df.loc[df['Filename']=='/home/plankton/Results/Peru/crops/17_03_2020/M3/CUSCO-Peru-2020_M3_PISCO20200317_151450.0700_1.215bar_0.jpg', ['avg_area']]=copepod_avg_area[0]
print(df)

       ID                                           Filename     x     y   w  \
0   18707  /home/plankton/Results/Peru/crops/17_03_2020/M...  1035  2295  14   
1   18708  /home/plankton/Results/Peru/crops/17_03_2020/M...  2376  2032  16   
2   18709  /home/plankton/Results/Peru/crops/17_03_2020/M...  1525  2020  18   
3   18710  /home/plankton/Results/Peru/crops/17_03_2020/M...  1359  1762  29   
4   18711  /home/plankton/Results/Peru/crops/17_03_2020/M...  1884  1565  14   
5   18712  /home/plankton/Results/Peru/crops/17_03_2020/M...   725  1564  18   
6   18713  /home/plankton/Results/Peru/crops/17_03_2020/M...  2433  1444  16   
7   18714  /home/plankton/Results/Peru/crops/17_03_2020/M...  1906  1418  29   
8   18715  /home/plankton/Results/Peru/crops/17_03_2020/M...   508  1294  16   
9   18716  /home/plankton/Results/Peru/crops/17_03_2020/M...  1018  1083  13   
10  18717  /home/plankton/Results/Peru/crops/17_03_2020/M...  2138   665  54   
11  18718  /home/plankton/Results/Peru/c